In [24]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [25]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [26]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [27]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [28]:
tokens = tokenizer.encode('I like this video , but it can be better', return_tensors='pt')

In [29]:
result = model(tokens)


In [30]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.6972, -1.0546,  1.5421,  1.8456,  0.1639]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [31]:
result.logits

tensor([[-2.6972, -1.0546,  1.5421,  1.8456,  0.1639]],
       grad_fn=<AddmmBackward0>)

In [32]:
result.logits
int(torch.argmax(result.logits))+1

4

In [33]:
r = requests.get('https://www.youtube.com/watch?v=WzPIkZi3v2g')
soup = BeautifulSoup(r.text, 'html.parser')


In [34]:
soup

<!DOCTYPE html>
<html darker-dark-theme="" darker-dark-theme-deprecate="" lang="en" style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" system-icons="" typography="" typography-spacing=""><head><script data-id="_gd" nonce="uUmIVRPBH16ZXUN1xqdY6g">window.WIZ_global_data = {"MuJWjd":false,"nQyAE":{},"oxN3nb":{"1":false,"644029907":true,"1822726157":true}};</script><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="ApvK67ociHgr2egd6c2ZjrfPuRs8BHcvSggogIOPQNH7GJ3cVlyJ1NOq/COCdj0+zxskqHt9HgLLETc8qqD+vwsAAABteyJvcmlnaW4iOiJodHRwczovL3lvdXR1YmUuY29tOjQ0MyIsImZlYXR1cmUiOiJQcml2YWN5U2FuZGJveEFkc0FQSXMiLCJleHBpcnkiOjE2OTUxNjc5OTksImlzU3ViZG9tYWluIjp0cnVlfQ==" http-equiv="origin-trial"/><script nonce="uUmIVRPBH16ZXUN1xqdY6g">var ytcfg={d:function(){return window.yt&&yt.config_||ytcfg.data_||(ytcfg.data_={})},get:function(k,o){return k in ytcfg.d()?ytcfg.d()[k]:o},set:function(){var a=arguments;if(a.length>1)ytcfg.d()[a[0]]=a[1];else{var k;for(k in a[0])ytcfg.d()[k]=a

In [35]:
soup.text

'White People & Indian Food (The Kohinoor story) | Standup Comedy Special by Raunaq Rajani - YouTubeAboutPressCopyrightContact usCreatorsAdvertiseDevelopersTermsPrivacyPolicy & SafetyHow YouTube worksTest new featuresNFL Sunday Ticket© 2024 Google LLC'

In [36]:
regex = re.compile('.*yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap".*')
results = soup.find_all('span', {'class': regex})
reviews = [result.text for result in results]


In [37]:
pip install google-api-python-client


In [38]:
from googleapiclient.discovery import build

# Set your API key here
api_key ='YOUR API KEY'

# Create a YouTube API service
youtube = build('youtube', 'v3', developerKey=api_key)

# Function to fetch comments from a YouTube video
def get_video_comments(video_id):
    # Requesting comments from the YouTube API
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=100,
        textFormat="plainText"
    )

    response = request.execute()

    comments = []

    # Extracting comments from the response
    while request:
        response = request.execute()

        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)

        # Check if there are more pages of comments
        if 'nextPageToken' in response:
            request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=100,
                textFormat="plainText",
                pageToken=response['nextPageToken']
            )
        else:
            break

    return comments

# Example usage: Replace with your video ID
video_id = "fIMiL9ZkWqs"
comments = get_video_comments(video_id)

# Print the comments
for idx, comment in enumerate(comments):
    print(f"{idx+1}: {comment}")


1: This record will never be broken
2: Where is Jack J97 ???
3: Piyush Joshi gaming 😎 is crazy 🤪 all thought ronando has 90 minute😅
4: Cr7🔥❤️🇧🇩
5: Maine Likhne wala Thaaa Piyush Joshi Ka Par aapnee Top 3 Mai rakh diya baad mai dekha
6: The thumbnail is genius but not honest
7: Virat kohli na bna diya na YouTube hake ho gay ga 🎉
8: I think it will be faster for Elon Musk
9: Then where is mizanur rahman Azhari??
10: Where is jackspedicey 2? That should be #1!
11: Johnson Carol Rodriguez Brenda Walker Daniel
12: Inshallah Amazon ka free course upload krna Wala hu 20 September ko sub subscribe kr la
13: @vjsidhuvlogs missed
14: Don't talk about fastest ,Talk about what happened after that Ronaldo is not going to cross 150 million, Where Mr Beast is going to Cross 400 million in 1 year..
15: Don't talk about fastest, just talk about what happened after that Ronaldo is not going to cross 150 million where Mr Beast will Cross 400 million within 1 year..
16: Cr7
17: Tomi11 😢
18: 100m Subscribe

In [39]:
import pandas as pd
import numpy as np

In [40]:
# Create a DataFrame from the comments
df = pd.DataFrame(np.array(comments), columns=["Comment"])

# Print the DataFrame
print(df)
# df = pd.DataFrame(np.array(reviews), columns=['review'])

                                                Comment
0                      This record will never be broken
1                                 Where is Jack J97 ???
2     Piyush Joshi gaming 😎 is crazy 🤪 all thought r...
3                                              Cr7🔥❤️🇧🇩
4     Maine Likhne wala Thaaa Piyush Joshi Ka Par aa...
...                                                 ...
1357  It’s wrong @Mizanur Rahman azhari got his 1 mi...
1358                                             Suiiii
1359                    Fastest 20,30,40,50 million pls
1360                                                 Hi
1361                             Interesting video bhia

[1362 rows x 1 columns]


In [41]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [42]:
sentiment_score(df['Comment'].iloc[1])

1

In [43]:
df['sentiment'] = df['Comment'].apply(lambda x: sentiment_score(x[:512]))

In [44]:
df

,Comment,sentiment
0,This record will never be broken,5
1,Where is Jack J97 ???,1
2,Piyush Joshi gaming 😎 is crazy 🤪 all thought r...,1
3,Cr7🔥❤️🇧🇩,5
4,Maine Likhne wala Thaaa Piyush Joshi Ka Par aa...,3
...,...,...
1357,It’s wrong @Mizanur Rahman azhari got his 1 mi...,1
1358,Suiiii,3
1359,"Fastest 20,30,40,50 million pls",5
1360,Hi,5


In [45]:
# Calculate the average sentiment score
average_sentiment = df['sentiment'].mean()

In [46]:
int(average_sentiment)

2